In [8]:
!date
import pandas as pd
import numpy as np
import sys
sys.path.append('../GOCAM_Project/dev')

import rpy2
from rpy2.robjects.packages import importr
BiasedUrn = importr('BiasedUrn')

import utils
import enrich
import numba

pd.options.display.max_colwidth = 100
import tqdm

Fri Dec 23 11:25:30 PST 2022


# Trying ncHGT on the data directly

P97 with standard method

In [2]:
df_s = enrich.enrich_wrapper('P97.csv','uniprot',method='set',show_significant=False)
np.sum(df_s['pval (uncorrected)'].values < 5e-03)

25

In [17]:
df_nc = enrich.enrich_wrapper('P97.csv','uniprot',method='ncHGT',show_significant=False)
df_nc[df_nc.title.apply(lambda x: 'PP2A' in x)]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
69,"PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome",0.006525,8,13,"[sset:PIP5K1A/B, sset:Activated SRC,LCK,EGFR,INSR, sset:PI4K2A/2B, sset:PP2A-subunit A, sset:Act...",http://model.geneontology.org/R-HSA-6811558


In [3]:
df_s[df_s.title.apply(lambda x: 'RHOA GTPase cycle - Reactome' in x)]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
35,RHOA GTPase cycle - Reactome,0.007182,2,2,"[sset:RHOA GEFs, sset:RHOA GAPs]",http://model.geneontology.org/R-HSA-8980692


In [4]:
gene_list, filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_ff  = enrich.enrich_wrapper('P97.csv','uniprot',method='standard',show_significant=False,return_all =True)
len(filtered_list)

270

In [15]:
#df_ff['ncHGT'] = df_ff.apply(lambda x: do_ncHGT_new( x['# genes in list'], x['url']), axis=1)
df_ff[df_ff.title.apply(lambda x: 'PP2A' in x)]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
198,"PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome",0.218219,8,115,"[P00533, P30154, P62714, Q9BTU6, Q99755, P35568, Q8NCE2, O14944]",http://model.geneontology.org/R-HSA-6811558


In [10]:
setID2members = utils.csv2dict('../data/setID2members.csv')
gocam2ID = utils.csv2dict('../data/gocam2ID_mouse.csv')
id2g_ff = utils.csv2dict('../data/ID2gocam_mouse_ff.csv')
Dict = utils.csv2dict('../data/ID2gocam_mouse.csv')
genes = np.array([*id2g_ff.keys()])
N = 270
size = 1000000
temp_gocam_keys = gocam2ID.keys()
gc_array = {}
for gc in temp_gocam_keys:
    gc_array[gc]=np.zeros(30)

iterator = tqdm.tqdm(range(size), position=0, leave=True)
for i in iterator:
    uni_list = np.random.choice(genes, size = N, replace= False)
    not_in_a_set, sets, setID2members_input_uni = enrich.get_sets(uni_list)

    filtered_out1, set_list_filtered = enrich.filter_gene_list(sets,Dict)
    filtered_out2, gene_list_filtered = enrich.filter_gene_list(uni_list, Dict) #clean gene_list to only include genes in the gocam

    filtered_list = gene_list_filtered + set_list_filtered

    counts = enrich.count_genes(filtered_list, Dict)
    
    for gc, k_array in gc_array.items():
        k = 0
        if gc in counts:
            k = len(counts.get(gc))
        k_array[k] +=1
    


100%|██████████████████████████████| 1000000/1000000 [1:57:58<00:00, 141.27it/s]


In [24]:
def get_pval(k,gc,gc_array,size):
    pval = np.sum(gc_array.get(gc)[k:])/size
    return pval

In [37]:
size = 1000000
alpha = .05
show_significant = True
background_num_gocams = len(gocam2ID)
df_sim = df_nc.copy()
df_sim['pval (uncorrected)']=df_sim.apply(lambda x: get_pval(x['# genes in list'], x['url'],gc_array,size), axis =1)

df_sim.sort_values('pval (uncorrected)',inplace=True)
df_sim.reset_index(drop=True, inplace=True)
df_sim['FDR_val'] = (df_sim.index+1)*alpha/background_num_gocams
df_sim['Less_than'] = (df_sim['pval (uncorrected)'] < df_sim['FDR_val'])
index = df_sim.Less_than.where(df_sim.Less_than==True).last_valid_index()
df_significant = df_sim
if (show_significant):
    df_significant = df_sim.loc[0:index].copy()
    if index == None:
        df_significant = pd.DataFrame(columns =['url', 'pval (uncorrected)', '# genes in list','#genes in gocam','shared gene products in gocam'])
df_sim = df_significant[['url','pval (uncorrected)', '# genes in list', '#genes in gocam','shared gene products in gocam']].copy()
df_sim

,url,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam
0,http://model.geneontology.org/R-HSA-176974,0.000000,6,7,"[P25205, Q14566, P33991, P49736, P33993, P33992]"
1,http://model.geneontology.org/R-HSA-69017,0.000003,14,65,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, P24941, Q92530, Q16763, O00762,..."
2,http://model.geneontology.org/R-HSA-174084,0.000004,13,59,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q16763, O00762, P51965,..."
3,http://model.geneontology.org/R-HSA-68867,0.000006,13,60,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q16763, O00762, P51965,..."
4,http://model.geneontology.org/R-HSA-187577,0.000007,12,49,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, P24941, Q92530, P61024, sset:p2..."
5,http://model.geneontology.org/R-HSA-174154,0.000008,13,62,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q16763, O00762, P51965,..."
6,http://model.geneontology.org/R-HSA-174178,0.000008,13,62,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q16763, O00762, P51965,..."
7,http://model.geneontology.org/R-HSA-174184,0.000008,13,62,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q16763, O00762, P51965,..."
8,http://model.geneontology.org/R-HSA-2467813,0.000009,13,63,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, O00487, Q92530, Q16763, O00762, P51965,..."
9,http://model.geneontology.org/R-HSA-176407,0.000021,11,49,"[Q13200, P43686, P62195, P62191, O00231, P35998, Q9UNM6, P06493, O00487, P24941, Q92530]"


In [21]:
k_experimental_array = 

23.471999999999994

In [13]:
utils.dict2csv(gc_array, 'sim_results_1000000.csv')

In [ ]:
gc = 'http://model.geneontology.org/R-HSA-6811558'
M = len(id2g)
w_M = 1
w, m = ncHGT.make_initial_vectors(gocam2ID,setID2members, gc, M,w_M)
w = w[:-1] #trim off background bin
XT = ncHGT.enumerate_possibilities(w,0,np.zeros(shape=(1,len(w)))) #note that we use w, not m, as the argument

In [ ]:
#filter XT to only include the region of the sample space >= k (which is what we want to sum probabilities over)
mask1 = (np.sum(XT==True, axis=1) >= k) #XT == True will convert it to a binary (ignoring repeated sampling)
XT = XT[mask1]


In [ ]:
results = enrich.enrich_wrapper('P97.csv','uniprot',method='ncHGT',alpha=.15)
results[results.title.apply(lambda x: 'PI5P, PP2A' in x)]

In [ ]:
background_num_gocams = 1100
alpha = .05
test = df_ff.copy()
test.sort_values('pval (uncorrected)',inplace=True)
test.reset_index(drop=True, inplace=True)
test['FDR_val'] = (test.index+1)*alpha/background_num_gocams
test['Less_than'] = (test['pval (uncorrected)'] < test['FDR_val'])
index = test.Less_than.where(test.Less_than==True).last_valid_index()
index

In [ ]:
background_num_gocams = 1100
alpha = .05
test = df_ff.copy()
test.sort_values('ncHGT',inplace=True)
test.reset_index(drop=True, inplace=True)
test['FDR_val'] = (test.index+1)*alpha/background_num_gocams
test['Less_than'] = (test['ncHGT'] < test['FDR_val'])
index = test.Less_than.where(test.Less_than==True).last_valid_index()
index

In [ ]:
.05*67/1100

In [ ]:
df_ff[df_ff.title.apply(lambda x: 'PI5P' in x)]

In [ ]:
test[0:67]